<a id='top'></a> 

## </a>Jump to [bottom](#bottom)

# RCA fish species and Temperature site attributes
 * Add HUC 12 and elevation data to temperature sites
 * Add AWC species and Northern pike data to RCA
  * all life stages/categories
 * Elevation stats for all RCAs
   * Mean min max
   * Mean elevation of contributing area
 * Percent Glacier cover calculation along flow acc network

### Import modules and set environments


In [1]:
import arcpy, os, zipfile, requests, datetime, sys, time, traceback
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

from datetime import datetime
today = datetime.now()
#Make the time stamp.  
time_stamp = '{:%d%m%Y}'.format(today)
print(time_stamp)
path = os.getcwd() # temporary working directories and geodatabases will be created here
print (path)

07022020
C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes


### Set variables
 * All input data are stored on the T: so no local data are needed
  * Temporary working directors are created in the current working directory listed above
    * Set path = "desired working folder" if necessary

In [80]:
import datetime

### Variable Names/identifiers ###

region = "Kenai" #region of interest - used to name temporary directories
outgdbname = region + ".gdb" #Name of output gdb
outdirname = region + "_Attributes_" #Location to save outputs
identifierfield = "reachid" #Name of field storing Catchment/RCA field in input stream layer ex. NHDPlus = "NHDPLusID", rca streams = "reachid"

### Network Data ###

instudy =  r"T:\\Aquatic\\KFHP\\Landscape_Metrics_DM\\Kenai\\Source_Data\\Kenai_Source.gdb\\kenai_studyarea" #Polygon defining study area, used to extract nlcd raster
instreams = r"T:\\Aquatic\\KFHP\\Landscape_Metrics_DM\\Kenai\\Source_Data\\Kenai_Source.gdb\\kenai_rca_reaches" #Streams with RCA code
inrca =  r"T:\\Aquatic\\KFHP\\Landscape_Metrics_DM\\Kenai\\Source_Data\\Kenai_Source.gdb\\kenai_rcas" #RCAs for region of interest
awc_events = "T:\\Aquatic\\KFHP\\AWC\\2018\\2018GDB_statewide.gdb\\awcEventArcs" # AWC events 
tempsites = "T:\\Aquatic\\KFHP\\Landscape_Metrics_DM\\Kenai\\Source_Data\\Kenai_Source.gdb\\tempsites_Kenai" # Temperature locations
huc12 = r"T:\\Aquatic\\KFHP\\Landscape_Metrics_DM\\Kenai\\Source_Data\\WBDHU12.shp" # HUC12 from NHD gdb

#flowrast =  r"T:\\Aquatic\\KFHP\\Landscape_Metrics_DM\\Kenai\\Source_Data\\Kenai_StrBrn_ad8ContribArea.tif" #flow accumulation raster for region of interest 
flowdirrast =  r"T:\\Aquatic\\KFHP\\Landscape_Metrics_DM\\Kenai\\Source_Data\\Kenai_StrBrn_d8flowdir.tif" #flow direction raster for region of interest
dem = "T:\\Aquatic\\KFHP\\TauDEM_Outputs\\Kenai\\Tau_out\\Kenai_StrBrn_2Int.tif" #DEM converted to Integer that matches extent and cell size of flow dir
orig_streamrast = r"T:\\Aquatic\\KFHP\\Landscape_Metrics_DM\\Kenai\\Source_Data\\Kenai_StrBrn_src.tif" #stream grid from synthetic stream network

identname = region + "_NLCD_Identity"#name of output Identity
tabtablename = region + "_NLCD_Tab_Int"#Name for output tabulate intersection table

#### Final product output location ####
networkgdb = r'T:\\Aquatic\\KFHP\\Geodatabases\\Kenai.gdb' # Location of shared geodatabase on network for final products
networkgdbfd = r'T:\\Aquatic\\KFHP\\Geodatabases\\Kenai.gdb\\Hydrology' # location of Hydrology feature dataset 

### Generated variables ###

streamdesc = arcpy.Describe(instreams)
streams_name = streamdesc.name
studydesc = arcpy.Describe(instudy)
study_name = studydesc.name
identname = streams_name + "_NLCD_Identity"#name of output Identity
tabtablename = streams_name + "_NLCD_Tab_Int"#Name for output tabulate intersection table

time = datetime.datetime.now()
print ("Variables set on", time)



Variables set on 2020-02-07 16:21:51.249202


### Create temporary folder and gdb

In [3]:
dirname = outdirname + str(time_stamp)
temp_dir = os.path.join(path,dirname)
ziploc = os.path.join(temp_dir, 'zips')
extractloc = os.path.join(temp_dir, 'extracts')

if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)
    os.makedirs(ziploc)
    os.makedirs(extractloc)        


outcheck = os.path.join(temp_dir,outgdbname)

if os.path.exists(outcheck):
    print ('Output location already exists', outcheck)
    outgdb = outcheck
if not os.path.exists(outcheck):
    print('Creating output GDB')
    tempgdb = arcpy.CreateFileGDB_management(temp_dir,outgdbname)
    print ('Output geodatabase created at',outcheck)
    outgdb = tempgdb.getOutput(0)

Creating output GDB
Output geodatabase created at C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes\Kenai_Attributes_07022020\Kenai.gdb


In [8]:
arcpy.env.workspace = outgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

indata = []
copylist = []
indata = [instudy,instreams,inrca,tempsites,awc_events]

studdesc = arcpy.Describe(instudy)
arcpy.CopyFeatures_management(instudy,studdesc.name)
studycopy = os.path.join(studdesc.path,studdesc.name)
copylist.append(studycopy)

reachdesc = studdesc = arcpy.Describe(instreams)
arcpy.CopyFeatures_management(instreams,reachdesc.name)
reachcopy = os.path.join(reachdesc.path,reachdesc.name)
copylist.append(reachcopy)

rcadesc = arcpy.Describe(inrca)
arcpy.CopyFeatures_management(inrca,rcadesc.name)
rcacopy = os.path.join(rcadesc.path,rcadesc.name)
copylist.append(rcacopy)

tempdesc = arcpy.Describe(tempsites)
arcpy.CopyFeatures_management(tempsites,tempdesc.name)
tempcopy = os.path.join(tempdesc.path,tempdesc.name)
copylist.append(tempcopy)

awcdesc = arcpy.Describe(awc_events)
arcpy.CopyFeatures_management(awc_events,awcdesc.name)
awccopy = os.path.join(awcdesc.path,awcdesc.name)
copylist.append(awccopy)   

print (copylist)

['T:\\\\Aquatic\\\\KFHP\\\\Landscape_Metrics_DM\\\\Kenai\\\\Source_Data\\\\Kenai_Source.gdb\\kenai_studyarea', 'T:\\\\Aquatic\\\\KFHP\\\\Landscape_Metrics_DM\\\\Kenai\\\\Source_Data\\\\Kenai_Source.gdb\\kenai_rca_reaches', 'T:\\\\Aquatic\\\\KFHP\\\\Landscape_Metrics_DM\\\\Kenai\\\\Source_Data\\\\Kenai_Source.gdb\\kenai_rcas', 'T:\\Aquatic\\KFHP\\Landscape_Metrics_DM\\Kenai\\Source_Data\\Kenai_Source.gdb\\tempsites_Kenai', 'T:\\Aquatic\\KFHP\\AWC\\2018\\2018GDB_statewide.gdb\\awcEventArcs']


## Begin Geospatial operations

### Join HUC12 data to RCA
     * Format RCAs
      * Drop all fields other than rca_id
      * Recalculate elevation (min-mean-max)
      * Recalculate contributing area
      

### Largest overlap Spatial Join
 * code from <a href = "https://www.arcgis.com/home/item.html?id=e9cccd343bf84916bda1910c31e5eab2">Largest Overlap</a>
 


In [9]:
# Main function, all functions run in SpatialJoinOverlapsCrossings
def SpatialJoinLargestOverlap(target_features, join_features, out_fc, keep_all, spatial_rel):
    if spatial_rel == "largest_overlap":
        # Calculate intersection between Target Feature and Join Features
        intersect = arcpy.analysis.Intersect([target_features, join_features], "in_memory/intersect", "ONLY_FID")
        # Find which Join Feature has the largest overlap with each Target Feature
        # Need to know the Target Features shape type, to know to read the SHAPE_AREA oR SHAPE_LENGTH property
        geom = "AREA" if arcpy.Describe(target_features).shapeType.lower() == "polygon" and arcpy.Describe(join_features).shapeType.lower() == "polygon" else "LENGTH"
        fields = ["FID_{0}".format(os.path.splitext(os.path.basename(target_features))[0]),
                  "FID_{0}".format(os.path.splitext(os.path.basename(join_features))[0]),
                  "SHAPE@{0}".format(geom)]
        overlap_dict = {}
        with arcpy.da.SearchCursor(intersect, fields) as scur:
            for row in scur:
                try:
                    if row[2] > overlap_dict[row[0]][1]:
                        overlap_dict[row[0]] = [row[1], row[2]]
                except:
                    overlap_dict[row[0]] = [row[1], row[2]]

        # Copy the target features and write the largest overlap join feature ID to each record
        # Set up all fields from the target features + ORIG_FID
        fieldmappings = arcpy.FieldMappings()
        fieldmappings.addTable(target_features)
        fieldmap = arcpy.FieldMap()
        fieldmap.addInputField(target_features, arcpy.Describe(target_features).OIDFieldName)
        fld = fieldmap.outputField
        fld.type, fld.name, fld.aliasName = "LONG", "ORIG_FID", "ORIG_FID"
        fieldmap.outputField = fld
        fieldmappings.addFieldMap(fieldmap)
        # Perform the copy
        arcpy.conversion.FeatureClassToFeatureClass(target_features, os.path.dirname(out_fc), os.path.basename(out_fc), "", fieldmappings)
        # Add a new field JOIN_FID to contain the fid of the join feature with the largest overlap
        arcpy.management.AddField(out_fc, "JOIN_FID", "LONG")
        # Calculate the JOIN_FID field
        with arcpy.da.UpdateCursor(out_fc, ["ORIG_FID", "JOIN_FID"]) as ucur:
            for row in ucur:
                try:
                    row[1] = overlap_dict[row[0]][0]
                    ucur.updateRow(row)
                except:
                    if not keep_all:
                        ucur.deleteRow()
        # Join all attributes from the join features to the output
        joinfields = [x.name for x in arcpy.ListFields(join_features) if not x.required]
        arcpy.management.JoinField(out_fc, "JOIN_FID", join_features, arcpy.Describe(join_features).OIDFieldName, joinfields)

In [11]:
arcpy.env.workspace = outgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

hucjoinname = region + "_rca_huc12_largeover_sj"
huccjoin = SpatialJoinLargestOverlap(rcacopy, huc12, hucjoinname, keep_all = "TRUE", spatial_rel = "largest_overlap" ) #Code from ESRI 

joindrop = []

awcjoin = os.path.join(outgdb+ "\\" + hucjoinname)

for field in arcpy.ListFields(awcjoin):
    joindrop.append(field.name)

keepfields = ['OBJECTID', 'Shape', 'rca_id','Shape_Length', 'Shape_Area','HUC12','Name']
for field in keepfields:   
    joindrop.remove(field)

arcpy.DeleteField_management(awcjoin, joindrop)

for field in arcpy.ListFields(awcjoin):
    print (field.name)

OBJECTID
Shape
rca_id
Shape_Length
Shape_Area
HUC12
Name


### Remove all fields from reaches except reach id and shape fields

In [12]:
arcpy.env.workspace = outgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

reachfields = []
for field in arcpy.ListFields(reachcopy):
    reachfields.append(field.name)

print (reachfields)
print ("")

keepfields = ['OBJECTID','OBJECTID_1', 'Shape', 'reachid','Shape_Length']
for field in keepfields:   
    reachfields.remove(field)

arcpy.DeleteField_management(reachcopy, reachfields)

for field in arcpy.ListFields(reachcopy):
    print (field.name)

['OBJECTID_1', 'Shape', 'OBJECTID', 'Id', 'Shape_Leng', 'rid', 'reachid', 'Z_Min', 'Z_Max', 'Z_Mean', 'SLength', 'slope_P', 'slope_D', 'Shape_Length']

OBJECTID_1
Shape
OBJECTID
reachid
Shape_Length


### Fish data
 * Join 5 salmon species and with lifestage data to each RCA
  * Format needs to be 3 columns for each species Spec/Lstag
  
 * Identity awc with rca first then spatial join and concatenate or selectby and populate fields

### Identity awc with rca

In [13]:
arcpy.env.workspace = outgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

awcidentname = region + "_awc_rca_IDENTITY"
awcclip = arcpy.Clip_analysis(awccopy,awcjoin,"tempclip")
awcident = arcpy.Identity_analysis(awcclip,awcjoin,awcidentname,"ALL")

print (awcident)

C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes\Kenai_Attributes_07022020\Kenai.gdb\Kenai_awc_rca_IDENTITY


### Add fish species/life stage fields and delete join fields

In [14]:
arcpy.env.workspace = outgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

lifestages = ['r','s','p']
species = ['K','CH','CO','P','S']
cols = []
for s in species:
    for l in lifestages:
        colname = str(s) + "_" + str(l)
        cols.append(colname)
print(cols)

for field in cols:
    arcpy.AddField_management(awcjoin,field,"SHORT") #add awcfields to rcas

deletefields = ['Join_Count','TARGET_FID','Join_Count_1','TARGET_FID_1','JOIN_FID']
arcpy.DeleteField_management(awcjoin,deletefields)

['K_r', 'K_s', 'K_p', 'CH_r', 'CH_s', 'CH_p', 'CO_r', 'CO_s', 'CO_p', 'P_r', 'P_s', 'P_p', 'S_r', 'S_s', 'S_p']


<Result 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\Watershed_Attributes\\Kenai\\Kenai_attributes\\Kenai_Attributes_07022020\\Kenai.gdb\\Kenai_rca_huc12_largeover_sj'>

### Query AWC events and calculate presence, spawning, and rearing attributes for RCAs from AWC/RCA identity
 * SPECIES
  * K: CHINOOK
  * CH: CHUM
  * CO: COHO
  * P: PINK
  * S: SOCKEYE
 
 
 * LIFE STAGE
  * r: rearing
  * s: spawning
  * p: presence - Presence has been calculated such that p = 1 where present or rearing or spawning as recorded in AWC

In [15]:
from time import strftime
start = datetime.datetime.now()

print( "Start script: " + strftime("%Y-%m-%d %H:%M:%S"))

fields = ['K_r', 'K_s', 'K_p', 'CH_r', 'CH_s', 'CH_p', 'CO_r', 'CO_s', 'CO_p', 'P_r', 'P_s', 'P_p', 'S_r', 'S_s', 'S_p']

### Chinook Selections ###
# "SPECIES = 'K' And LSTAGE = 'r' OR SPECIES = 'K' And LSTAGE = 's' OR SPECIES = 'K' And LSTAGE = 'p'" # Modify presence to include all

akr = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'K' And LSTAGE = 'r'")
rkr = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', akr)
arcpy.CalculateField_management(rkr,'K_r', 1)

aks = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'K' And LSTAGE = 's'")
rks = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', aks)
arcpy.CalculateField_management(rks,'K_s', 1)

akp = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'K' And LSTAGE = 'r' OR SPECIES = 'K' And LSTAGE = 's' OR SPECIES = 'K' And LSTAGE = 'p'")
rkp = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', akp)
arcpy.CalculateField_management(rkp,'K_p', 1)

### Chum Selections ###
achr = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'CH' And LSTAGE = 'r'")
rchr = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', achr)
arcpy.CalculateField_management(rchr,'CH_r', 1)

achs = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'CH' And LSTAGE = 's'")
rchs = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', achs)
arcpy.CalculateField_management(rchs,'CH_s', 1)

achp = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'CH' And LSTAGE = 'r' OR SPECIES = 'CH' And LSTAGE = 's' OR SPECIES = 'CH' And LSTAGE = 'p'")
rchp = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', achp)
arcpy.CalculateField_management(rchp,'CH_p', 1)

### Coho Selection ###
acor = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'CO' And LSTAGE = 'r'")
rcor = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', acor)
arcpy.CalculateField_management(rcor,'CO_r', 1)

acos = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'CO' And LSTAGE = 's'")
rcos = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', acos)
arcpy.CalculateField_management(rcos,'CO_s', 1)

acop = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'CO' And LSTAGE = 'r' OR SPECIES = 'CO' And LSTAGE = 's' OR SPECIES = 'CO' And LSTAGE = 'p'")
rcop = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', acop)
arcpy.CalculateField_management(rcop,'CO_p', 1)

### Pink Selection ###
apr = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'P' And LSTAGE = 'r'")
rpr = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', apr)
arcpy.CalculateField_management(rpr,'P_r', 1)

aps = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'P' And LSTAGE = 's'")
rps = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', aps)
arcpy.CalculateField_management(rps,'P_s', 1)

app = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'P' And LSTAGE = 'r' OR SPECIES = 'P' And LSTAGE = 's' OR SPECIES = 'P' And LSTAGE = 'p'")
rpp = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', app)
arcpy.CalculateField_management(rpp,'P_p', 1)

### Sockeye Selection ###
asr = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'S' And LSTAGE = 'r'")
rsr = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', asr)
arcpy.CalculateField_management(rsr,'S_r', 1)

ass = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'S' And LSTAGE = 's'")
rss = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', ass)
arcpy.CalculateField_management(rss,'S_s', 1)

asp = arcpy.SelectLayerByAttribute_management(awcident, "NEW_SELECTION", "SPECIES = 'S' And LSTAGE = 'r' OR SPECIES = 'S' And LSTAGE = 's' OR SPECIES = 'S' And LSTAGE = 'p'")
rsp = arcpy.SelectLayerByLocation_management(awcjoin,'CONTAINS_CLEMENTINI', asp)
arcpy.CalculateField_management(rsp,'S_p', 1)

stop = datetime.datetime.now()
elapsed = stop - start
print( "End script: " + strftime("%Y-%m-%d %H:%M:%S") + " Runtime = " +  str(elapsed))


Start script: 2020-02-07 15:44:32
End script: 2020-02-07 15:44:45 Runtime = 0:00:13.009900


### Convert null values in fish species columns to zero

In [16]:
fieldList =  fields
with arcpy.da.UpdateCursor(awcjoin, fieldList) as cursor:
    fRange = range(len(fieldList)) # create an index 0 to the number of elements in fieldList - 1

    for row in cursor:
        

        # step through each field in the row by its index
        for index in fRange:
            if row[index] == None:
                row[index] = 0         #set null to zero
            
            
            cursor.updateRow(row)

print ("Finished")

Finished


### Reclassify Tau DEM flow direction to work with ESRI Flowaccumulation()

![Esri_flowdir](https://user-images.githubusercontent.com/36055691/74059969-7891d780-4995-11ea-9eeb-2ab8f31dbb31.jpg)


In [17]:
import arcpy
from arcpy.sa import *
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

start= datetime.datetime.now()
print ("Begin process", start)

arcpy.env.workspace = temp_dir
arcpy.env.overwriteOutput = True
spref = arcpy.SpatialReference("Alaska Albers Equal Area Conic")
arcpy.env.outputCoordinateSystem = spref


rastdesc2 = arcpy.Describe(flowdirrast)
print (rastdesc2.name)
rastname2 = rastdesc2.name
flowdircopy = Reclassify(flowdirrast, "Value", RemapValue([[1,1],[2,128],[3,64],[4,32],[5,16],[6,8],[7,4],[8,2]]))
flowdircopy.save(rastname2)
flowdirscribe = arcpy.Describe(flowdircopy)
flowdirpath = os.path.join(flowdirscribe.path, flowdirscribe.name)

stop = datetime.datetime.now()
elapsed = stop - start
print ("Process complete ", elapsed)
print (flowdirpath)

Begin process 2020-02-07 15:44:48.042993
Kenai_StrBrn_d8flowdir.tif
Process complete  0:00:24.181861
C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes\Kenai_Attributes_07022020\Kenai_StrBrn_d8flowdir.tif


### Copy Integer DEM from TauDEM to local
 * outputs from TauDEM have Cellsize slightly smaller than original DEM source raster
   * This happened during the conversion of the orignal DEM from float to Integer
     

In [18]:
arcpy.env.workspace = temp_dir
arcpy.env.snapRaster = flowdirpath
arcpy.env.extent = flowdirpath
arcpy.env.cellsize = flowdirpath

arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

demdescribe = arcpy.Describe(dem)
demname = temp_dir + "\\" + demdescribe.name
dem_copy = ExtractByMask(dem, flowdirpath)
dem_copy.save(demname)

print ("Processes Complete")

Processes Complete


### Extract stream src by reclassified flow dir
 * Probably unecessary but going to leave in for now to make sure all rasters are aligned

In [19]:
arcpy.env.workspace = temp_dir
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")
arcpy.env.snapRaster = flowdirpath
arcpy.env.extent = flowdirpath

streamdescribe = arcpy.Describe(orig_streamrast)
streamname = temp_dir + "\\" + streamdescribe.name
streamrast = ExtractByMask(orig_streamrast, flowdirpath)
streamrast.save(streamname)

print ("Extraction Complete")

Extraction Complete


### Check rasters
 * Compare extent, cellsize, projection

In [20]:
rastlist = [streamrast, flowdirpath, dem_copy]
for raster in rastlist:
    desc = arcpy.Describe(raster)

    
    raster_min = arcpy.Raster(raster).minimum
    raster_max = arcpy.Raster(raster).maximum
    raster_mean = arcpy.Raster(raster).mean
    extent = arcpy.Raster(raster).extent
    
    print("Raster name:      %s" % desc.name)
    print("Projection:      %s" % desc.SpatialReference.name)
    print("Compression Type: %s" % desc.compressionType)
    print("Raster Format:    %s" % desc.format)
    print("Height: %d" % desc.height)
    print("Width:  %d" % desc.width)
    print("Cellsize:  %f" % desc.meanCellHeight)
    print("Integer Raster: %s" % desc.isInteger)
    print("Raster stats: min = {:,.2f} max = {:,.2f} mean = {:,.2f}".format(raster_min,raster_max,raster_mean))
    print (extent)
    print ("")   

Raster name:      Kenai_StrBrn_src.tif
Projection:      NAD_1983_Alaska_Albers
Compression Type: LZW
Raster Format:    TIFF
Height: 18755
Width:  27650
Cellsize:  5.000000
Integer Raster: True
Raster stats: min = 0.00 max = 1.00 mean = 0.01
147477.63147271 1116160.30748062 285727.63147271 1209935.30748062 NaN NaN NaN NaN

Raster name:      Kenai_StrBrn_d8flowdir.tif
Projection:      NAD_1983_Alaska_Albers
Compression Type: LZW
Raster Format:    TIFF
Height: 18755
Width:  27650
Cellsize:  5.000000
Integer Raster: True
Raster stats: min = 1.00 max = 128.00 mean = 33.07
147477.63147271 1116160.30748062 285727.63147271 1209935.30748062 NaN NaN NaN NaN

Raster name:      Kenai_StrBrn_2Int.tif
Projection:      NAD_1983_Alaska_Albers
Compression Type: LZW
Raster Format:    TIFF
Height: 18755
Width:  27650
Cellsize:  5.000000
Integer Raster: True
Raster stats: min = 0.00 max = 1,974.00 mean = 521.31
147477.63147271 1116160.30748062 285727.63147271 1209935.30748062 NaN NaN NaN NaN



### Add Elevation data from DEM


In [21]:
from arcpy.sa import *
start = datetime.datetime.now()
arcpy.env.workspace = outgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

print( "Start script: " + strftime("%Y-%m-%d %H:%M:%S"))
print ("")

tablename = region + "_DEM_Zonal_Table"

print ("Calculating zonal statistics")
print("")

stop = datetime.datetime.now()  
elapsed = stop - start  

print ('Time to complete = ',elapsed)
print("")

ztable = ZonalStatisticsAsTable(awcjoin, 'OBJECTID', dem_copy, tablename, 'DATA', 'MIN_MAX_MEAN')

print ("Joining statistics to RCAs")
print("")

arcpy.JoinField_management(awcjoin, 'OBJECTID', ztable, 'OBJECTID_1', ['MIN','MAX','MEAN'])


stop = datetime.datetime.now()  
elapsed = stop - start  
print ('Time to complete = ',elapsed)


Start script: 2020-02-07 15:48:40

Calculating zonal statistics

Time to complete =  0:00:00.013962

Joining statistics to RCAs

Time to complete =  0:00:22.530401


In [22]:
import pandas as pd
pd.set_option('display.max_columns', None)  
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis import GIS
gis = GIS()
sdf = pd.DataFrame.spatial.from_featureclass(awcjoin)
sdf.head()


,OBJECTID,rca_id,HUC12,Name,K_r,K_s,K_p,CH_r,CH_s,CH_p,CO_r,CO_s,CO_p,P_r,P_s,P_p,S_r,S_s,S_p,MIN,MAX,MEAN,SHAPE
0,1,1.0,190203021001,Headwaters Trail Creek,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,224,231,227.496154,"{'rings': [[[267552.6314670332, 1187445.307535..."
1,2,2.0,190203021001,Headwaters Trail Creek,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,261,307,272.822581,"{'rings': [[[268957.6314515844, 1189145.307505..."
2,3,3.0,190203021001,Headwaters Trail Creek,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,265,438,300.738371,"{'rings': [[[269757.6314552445, 1189390.307459..."
3,4,5.0,190203021602,Middle Moose River,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,54,76,57.763341,"{'rings': [[[187427.63152244315, 1188305.30747..."
4,5,6.0,190203021607,Muskrat Lake,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,64,84,74.382605,"{'rings': [[[193722.63148560934, 1195020.30750..."


## Identify outlets for each RCA
  * Shrink RCAs by 15 meters shift outlet slighty upstream in order to avoid any errors that may smaller RCAs shifted into larger systems
 * Identify RCAs that were missed in the first operation and shrink by 8 meters
 * Identify any remaining RCAs missed by the first 2 operations and identify outlet
### Merge results together and remove any duplicates

### Create Flow Accumulation

In [23]:
from datetime import datetime
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")
arcpy.env.workspace = temp_dir
arcpy.env.snapRaster = flowdirpath 
arcpy.env.extent = flowdirpath

today = datetime.now()
# Make the time stamp.  
time_stamp = '{:%d%m%Y}'.format(today)

print(time_stamp)

import datetime
start = datetime.datetime.now()

print ("Begin process", start)
print("")

flowrastname = temp_dir + "\\" + region + "_flow_accumulation_" + time_stamp + ".tif"                

print ("Creating ",flowrastname)
print("")

flowacc = FlowAccumulation(flowdirpath,"",  'FLOAT', 'D8') # create flow acc from flow dir
flowacc.save(flowrastname)
flowaccdescribe = arcpy.Describe(flowacc)
flowaccpath = os.path.join(flowacc.path,flowacc.name)


stop = datetime.datetime.now()
elapsed = stop - start
print ("Process complete ", elapsed)


07022020
Begin process 2020-02-07 15:49:08.577757

Creating  C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes\Kenai_Attributes_07022020\Kenai_flow_accumulation_07022020.tif

Process complete  0:01:36.277028


### Shrink RCA by 15 meters
* Consider filtering out very small RCAs (those that would be dissolved with Inside Buffer) prior to running 

In [24]:
import datetime
arcpy.env.workspace = outgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

start = datetime.datetime.now()
print ("Begin Process", start)

inrca2 = awcjoin
buffval = -15 #shrink polygons by 15 meters
rcadesc = arcpy.Describe(inrca2)
buffabs = abs(buffval)
insidebuffname = rcadesc.name + "_InsideBuffer_" + str(buffabs)
shrinkrca = arcpy.Buffer_analysis(inrca2,insidebuffname,buffval)

print(shrinkrca)
stop = datetime.datetime.now()
elapsed = stop - start
print ("Process complete ", elapsed)

Begin Process 2020-02-07 15:50:44.875729
C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes\Kenai_Attributes_07022020\Kenai.gdb\Kenai_rca_huc12_largeover_sj_InsideBuffer_15
Process complete  0:00:53.603825


### Identify and reclassify stream grid using TauDEM stream source grid and rca stream reaches

In [25]:
from datetime import datetime

arcpy.env.workspace = temp_dir
arcpy.env.snapRaster = flowdirpath
arcpy.env.extent = flowdirpath
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")


today = datetime.now()
# Make the time stamp.  
time_stamp = '{:%d%m%Y}'.format(today)
import datetime
start = datetime.datetime.now()

print ("Begin process", start)
print("")


outname = temp_dir + "\\" + region + "_rcastream_src_maskextract_" + time_stamp + ".tif"
streamrast2 = ExtractByMask(streamrast, reachcopy)
streamrast2.save(outname)

streamname = temp_dir + "\\" + region + "_streams_raster_" + time_stamp + ".tif"
#streamcon = Con((Con(IsNull(streamrast),0,1) + streamrast2),0,1,"VALUE < 1") #Identify stream network from the two input stream grids and reclassify as 1 for stream and 0 everywhere else
streamcon = Con(IsNull(streamrast2), 0, streamrast2) #Reclassify null values in extracted stream src grid as 0
streamcon.save(streamname)

print (streamcon)

stop = datetime.datetime.now()
elapsed = stop - start

print ("Process complete ", elapsed)
print("")

Begin process 2020-02-07 15:51:38.502493

C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes\Kenai_Attributes_07022020\Kenai_streams_raster_07022020.tif
Process complete  0:00:33.414781



### Identify outlets using stream accumulation grid, shrunken buffer, and zonal statistics
 

In [26]:
start = datetime.datetime.now()
arcpy.env.workspace = temp_dir
arcpy.env.snapRaster = flowdirpath
arcpy.env.extent = flowdirpath
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")


print ("Creating stream flow accumulation raster")

streamflowname = temp_dir + "\\" + region + "_stream_accumulation.tif"                
streamflow = SetNull(streamcon,flowaccpath,"Value = 0") # Create flow accumulation raster along stream network only
streamflow.save(streamflowname)

print ("Creating max accumulation raster")

max_acczon = temp_dir +"//" + region + "_max_acc_zon_ALL.tif"
max_acc_zon = arcpy.sa.ZonalStatistics(shrinkrca, 'OBJECTID', streamflow, 'MAXIMUM', 'DATA') #identify max flow accumulation using shrunken rca raster
max_acc_zon.save(max_acczon)

print (max_acc_zon, "Created")
print("")
print ("Identifying highest accumulation cell for each catchment")

max_accatch = temp_dir +"//" + region + "_max_acc_catch_ALL.tif"
max_acc_catch = Con(streamflow == max_acc_zon,streamflow) # identify cell of highest accumulation for each rca along stream network
max_acc_catch.save(max_accatch)

print ("Catchment outlets Id'd")
print (max_acc_catch)

stop = datetime.datetime.now()
elapsed = stop - start  
print ('Time to complete = ',elapsed)


Creating stream flow accumulation raster
Creating max accumulation raster
C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes\Kenai_Attributes_07022020\Kenai_max_acc_zon_ALL.tif Created

Identifying highest accumulation cell for each catchment
Catchment outlets Id'd
C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes\Kenai_Attributes_07022020\Kenai_max_acc_catch_ALL.tif
Time to complete =  0:01:08.744366


### Convert to point
 * First set of outlets, some RCAs will be missed due to shrunken buffer

In [27]:
arcpy.env.workspace = outgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

start = datetime.datetime.now()

print ('Converting Max accumulation cells to points')
print ('')

convers_name = outgdb + "\\" + region + "_max_acc_outlet_FROM_ALL"
outlets = arcpy.RasterToPoint_conversion(max_acc_catch,convers_name)


print ('Conversion COMPLETE')
print ('')

stop = datetime.datetime.now()
elapsed = stop - start

print ('Time to complete =',elapsed)

Converting Max accumulation cells to points

Conversion COMPLETE

Time to complete = 0:00:04.850974


### Identify missing RCAs
 * Select missed RCAs using outlets produced from the first operation
  * Rerun code above to create second outlet feature class using an 8 meter Inside Buffer
 


In [28]:
arcpy.env.workspace = outgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

start = datetime.datetime.now()

print ("Selecting RCAs missed during buffer shrink")
print("")

missedout = arcpy.SelectLayerByLocation_management(awcjoin, 'INTERSECT', outlets, 1, 'NEW_SELECTION', 'INVERT') #Identify missed RCAs

buffval = -8 #shrink missed polygons by 8 meters
rcadesc = arcpy.Describe(inrca2)
buffabs = abs(buffval)
insidebuffname = rcadesc.name + "_InsideBuffer_" + str(buffabs)
shrinkrca2 = arcpy.Buffer_analysis(missedout, insidebuffname,buffval)

print ("Creating max accumulation raster")
print("")

max_acczon2 = temp_dir +"//" + region + "_max_acc_zon_ALL2.tif"
max_acc_zon2 = arcpy.sa.ZonalStatistics(shrinkrca2, 'OBJECTID', streamflow, 'MAXIMUM', 'DATA') #identify max flow accumulation using shrunken rca raster
max_acc_zon2.save(max_acczon2)

print ("Identifying highest accumulation cell for each catchment")
print("")

max_accatch2 = temp_dir +"//" + region + "_max_acc_catch_ALL2.tif"
max_acc_catch2 = Con(streamflow == max_acc_zon2,streamflow) # identify cell of highest accumulation for each rca along stream network
max_acc_catch2.save(max_accatch2)

print ("Catchment outlets Id'd")
print("")

print ('Converting Max accumulation cells to points')
print("")

convers_name = outgdb + "\\" + region + "_max_acc_outlet_FROM_ALL2"
outlets2 = arcpy.RasterToPoint_conversion(max_acc_catch2,convers_name)

print ('Conversion COMPLETE')
print('')

print("Merging outlets and deleting any duplicates")
print ("")

pointmergename = outgdb + "\\" + region + "_outlets_Merge1"
outletmerge = arcpy.Merge_management([outlets,outlets2 ],pointmergename)

print('')
print ('Merge COMPLETE')

stop = datetime.datetime.now()
elapsed = stop - start
print ('Time to complete =',elapsed)

Selecting RCAs missed during buffer shrink

Creating max accumulation raster

Identifying highest accumulation cell for each catchment

Catchment outlets Id'd

Converting Max accumulation cells to points

Conversion COMPLETE

Merging outlets and deleting any duplicates


Merge COMPLETE
Time to complete = 0:00:55.278051


### Run a third time without inside buffer on any missed outlets
 * Tend to be very small RCAs near the confluence of larger systems

In [29]:
arcpy.env.workspace = outgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")


start = datetime.datetime.now()

print ("Selecting RCAs missed during buffer shrink")
print("")

missedout2 = arcpy.SelectLayerByLocation_management(awcjoin, 'INTERSECT', outletmerge, 1, 'NEW_SELECTION', 'INVERT') #Identify missed RCAs
miss2 = arcpy.GetCount_management(missedout2)
print ("final selection of " + str(miss2) + " missed RCAs")

print ("Creating max accumulation raster")
print("")

max_acczon3 = temp_dir +"//" + region + "_max_acc_zon_ALL3.tif"
max_acc_zon3 = arcpy.sa.ZonalStatistics(missedout2, 'OBJECTID', streamflow, 'MAXIMUM', 'DATA') #identify max flow accumulation using shrunken rca raster
max_acc_zon3.save(max_acczon3)

print ("Identifying highest accumulation cell for each catchment")
print("")

max_accatch3 = temp_dir +"//" + region + "_max_acc_catch_ALL3.tif"
max_acc_catch3 = Con(streamflow == max_acc_zon3,streamflow) # identify cell of highest accumulation for each rca along stream network
max_acc_catch3.save(max_accatch3)

print ("Catchment outlets Id'd")
print("")

print ('Converting Max accumulation cells to points')
print("")

convers_name = outgdb + "\\" + region + "_max_acc_outlet_FROM_ALL3"
outlets3 = arcpy.RasterToPoint_conversion(max_acc_catch3,convers_name)

print ('Conversion COMPLETE')
print('')

print("Merging outlets")
print ("")

pointmergename2 = outgdb + "\\" + region + "_outlets_Merge2"
outletmerge2 = arcpy.Merge_management([outletmerge, outlets3],pointmergename2)

print('')
print ('Merge COMPLETE')

joinname = outgdb + "\\" + region + "_RCA_Outlets_" + time_stamp
idjoin2 = arcpy.SpatialJoin_analysis(outletmerge2, awcjoin, joinname, 'JOIN_ONE_TO_ONE', 'KEEP_ALL',"", 'INTERSECT')

print ("Join Complete...deleting duplicates")
print ("")

arcpy.DeleteIdentical_management(idjoin2,'SHAPE')

print ("Extracting flow accumulation to outlet")
print ("")

ExtractMultiValuesToPoints(idjoin2,[[streamflow, "Flow_Accumulation"]])

print ("Process complete")

count = arcpy.GetCount_management(idjoin2)

print (str(count) + " Outlets created")

stop = datetime.datetime.now()
elapsed = stop - start
print ('Time to complete =',elapsed)

Selecting RCAs missed during buffer shrink

final selection of 39 missed RCAs
Creating max accumulation raster

Identifying highest accumulation cell for each catchment

Catchment outlets Id'd

Converting Max accumulation cells to points

Conversion COMPLETE

Merging outlets


Merge COMPLETE
Join Complete...deleting duplicates

Extracting flow accumulation to outlet

Process complete
3333 Outlets created
Time to complete = 0:00:59.938259


### Calculate Mean Elevation for RCA + all upstream contributing area
 * Create elevation weighted flow accumulation and divide by flow accumulation
 * Extract to outlet

In [30]:
arcpy.env.workspace = temp_dir
arcpy.env.snapRaster = flowdirpath
arcpy.env.extent = flowdirpath
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

start = datetime.datetime.now()

print ("Begin process", start)
print("")

elevw8name = temp_dir + "\\" + region + "_elevation_weighted_flowacc.tif"
elevw8flow = FlowAccumulation(flowdirpath, dem_copy, 'FLOAT', 'D8') # create glacially weighted flow accumulation
elevw8flow.save(elevw8name)

print(elevw8flow)
print ("")

stop = datetime.datetime.now()
elapsed = stop - start

print ("Process complete ", elapsed)
print ("")
print ("Calculate Mean Elevation along stream network")
print ("")

start = datetime.datetime.now()

meanelevname = temp_dir + "\\" + region + "stream_MEAN_elev.tif"                
meanelev = SetNull(streamflow,(elevw8flow / flowaccpath ),"Value = 0") #Limit output to stream network
meanelev.save(meanelevname)

ExtractMultiValuesToPoints(idjoin2,[[meanelev, "Mean_Elev_Contributing_Area"]])

stop = datetime.datetime.now()
elapsed = stop - start

print ("Process complete ", elapsed)
print(meanelev)


Begin process 2020-02-07 15:55:20.824672

C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes\Kenai_Attributes_07022020\Kenai_elevation_weighted_flowacc.tif

Process complete  0:01:43.709087

Calculate Mean Elevation along stream network

Process complete  0:01:13.545438
C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes\Kenai_Attributes_07022020\Kenaistream_MEAN_elev.tif


### Create Final attributed RCA on local

<a id='deleteid'></a>

In [31]:
arcpy.env.workspace = outgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

rcadesc = arcpy.Describe(rcacopy)
name = rcadesc.name
rcacopyname = name +"_attributed_" + time_stamp
finalrca = arcpy.CopyFeatures_management(rcacopy, rcacopyname)

for field in arcpy.ListFields(finalrca):
    joindrop.append(field.name)
    
keepfields = ['OBJECTID', 'Shape', 'rca_id','Shape_Length', 'Shape_Area']
for field in keepfields:
    joindrop.remove(field)

arcpy.DeleteField_management(finalrca, joindrop)

tempdesc = arcpy.Describe(tempsites)
tempcopyname = name + "_temp_sites_attributed_" + time_stamp
tempfinal = arcpy.CopyFeatures_management(tempsites,tempcopyname)
ExtractMultiValuesToPoints(tempfinal,[[dem_copy, "Elevation_meters"]])

print("Extracting elevation at temperature site")
print(tempfinal)
print ("")

outletcopyname = region + "_rca_outlets_attributed_" + time_stamp
outcopy = arcpy.CopyFeatures_management(idjoin2, outletcopyname)
arcpy.DeleteIdentical_management(outcopy,'rca_id')
count = arcpy.GetCount_management(outcopy)

print (str(count) + " Outlets created")
print("")

count2 = arcpy.GetCount_management(finalrca)

print (str(count2) + " RCAs created in the " + str(region) + "study area")
print ("")

arcpy.JoinField_management(finalrca,"rca_id",outcopy, "rca_id")



Extracting elevation at temperature site
C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes\Kenai_Attributes_07022020\Kenai.gdb\kenai_rcas_temp_sites_attributed_07022020

3325 Outlets created

3325 RCAs created in the Kenaistudy area



<Result 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\Watershed_Attributes\\Kenai\\Kenai_attributes\\Kenai_Attributes_07022020\\Kenai.gdb\\kenai_rcas_attributed_07022020'>

### Recalculate slope percent reaches
 * (rise/run*100)

In [32]:
from arcpy.sa import * 
method = "BILINEAR"
prop = "Z_MIN;Z_MAX;Z_MEAN;SURFACE_LENGTH"
perc_calc = '(!Z_Max!-!Z_Min!)/!SLength!*100'

print ("Adding Surface information to rca reaches")
print ("")

arcpy.ddd.AddSurfaceInformation(reachcopy,dem_copy,prop,method)

print ("Adding Surface info fields")
print ("")

arcpy.AddField_management(reachcopy,"reach_length", "DOUBLE","","","", "Reach Length in meters")
arcpy.AddField_management(reachcopy,"reach_slope", 'DOUBLE',"","","","Reach slope in percent")
arcpy.CalculateField_management(reachcopy,"reach_slope", perc_calc)
arcpy.CalculateField_management(reachcopy,"reach_length", '!SLength!')




Adding Surface information to rca reaches

Adding Surface info fields



<Result 'T:\\\\Aquatic\\\\KFHP\\\\Landscape_Metrics_DM\\\\Kenai\\\\Source_Data\\\\Kenai_Source.gdb\\kenai_rca_reaches'>

## Calcute percent land cover metrics in 30meter buffered area

### Download NLCD
 * NLCD is currently the best available landcover dataset with coverage for the entire state

In [33]:
import datetime
wurl = r"https://prd-tnm.s3.amazonaws.com/StagedProducts/NLCD2011/Land_Cover/Alaska/ak_nlcd_2011_landcover_1_15_15.zip"
start = datetime.datetime.now() 
wname = wurl[-34:]
print ('Downloading',wname)

wzippath = str(ziploc) + '/'+ str(wname) #path to save download to plus name of download
# headers = {"Range": "bytes=0-100"}  # first 100 bytes
# rh= requests.get(wurl, headers)
# print(rh.status_code)
# print(rh.headers['content-type'])
# print(rh.encoding)
r = requests.get(wurl)
if not os.path.exists(wzippath):
    with open(wzippath,'wb') as f:
        f.write(r.content)
        
print('')
print ('ALL DOWNLOADS COMPLETE')
stop = datetime.datetime.now()  
elapsed = stop - start  
print ('Time to complete = ',elapsed)


ALL DOWNLOADS COMPLETE
Time to complete =  0:00:31.195683


### Unzip NLCD
#### Must have 7zip installed and located in the path below, change if necessary


In [34]:
### Unzip landcover data

import subprocess
os.chdir(ziploc)
for dir in os.listdir():
    if "ak_nlcd_2011" in dir:
        wzip = os.path.abspath(dir)
        print ('Unzipping ', wzip)
        #wzip = zipfile.ZipFile(file_name) # create zipfile object
        #wzip.extractall(extractloc) # extract file to dir
        #wzip.close() # close file
        #os.remove(file_name) # delete zipped file if required
        wuz = subprocess.call(r'"C:\Program Files\7-Zip\7z.exe" x ' + wzip + ' -o' + extractloc)
        print ('Finished extracting AK NLCD')
print('')
print ('Unzipping complete')
stop = datetime.datetime.now()  
elapsed = stop - start  
print ('Time to complete = ',elapsed)

Unzipping  C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes\Kenai_Attributes_07022020\zips\ak_nlcd_2011_landcover_1_15_15.zip
Finished extracting AK NLCD

Unzipping complete
Time to complete =  0:00:49.937943


### Download Glacier data - KENAI ONLY
 * approx 1 gb download as of 20200117

In [35]:
import datetime
wurl = r"http://www.glims.org/download/latest" #latest glims data
start = datetime.datetime.now() 
wname = "GLIMS_Data.zip"
print ('Downloading',wname)


wzippath = str(ziploc) + '/'+ str(wname) #path to save download to plus name of download
# headers = {"Range": "bytes=0-100"}  # first 100 bytes
# rh= requests.get(wurl, headers)
# print(rh.status_code)
# print(rh.headers['content-type'])
# print(rh.encoding)
r = requests.get(wurl)
if not os.path.exists(wzippath):
    with open(wzippath,'wb') as f:
        f.write(r.content)
        
print('')
print ('ALL DOWNLOADS COMPLETE')
stop = datetime.datetime.now()  
elapsed = stop - start  
print ('Time to complete = ',elapsed)

os.chdir(ziploc)
start = datetime.datetime.now()
for item in os.listdir():
    if "GLIMS" in item:
        file_name = os.path.abspath(item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(extractloc) # extract file to dir
        zip_ref.close() # close file
        #os.remove(file_name) # delete zipped file if required   
        print ('Unzipping..', file_name)
        print('')

print ('Unzipping complete')
stop = datetime.datetime.now()  
elapsed = stop - start  
print ('Time to complete = ',elapsed)


ALL DOWNLOADS COMPLETE
Time to complete =  0:05:19.005568
Unzipping.. C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes\Kenai_Attributes_07022020\zips\GLIMS_Data.zip

Unzipping complete
Time to complete =  0:00:18.996854


### Buffer by 30 meters and clip to study area
 * Buffers will overlap but this is not an issue
  * Adding text field to copy NHDPLUSID (Using Float field may cause issues with panda display) and add and calc shape area in square meters as a check

In [36]:
arcpy.env.workspace = outgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

reachdesc = arcpy.Describe(reachcopy)
streambuffname = reachdesc.name + "_buff30"
riv_buff30 = arcpy.Buffer_analysis(reachcopy,streambuffname,30, 'FULL', 'FLAT', 'NONE')
arcpy.AddField_management(riv_buff30,"Buff_Area_Sqm", 'DOUBLE')
arcpy.CalculateField_management(riv_buff30, 'Buff_Area_Sqm','!shape.area@squaremeters!')


<Result 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\Watershed_Attributes\\Kenai\\Kenai_attributes\\Kenai_Attributes_07022020\\Kenai.gdb\\kenai_rca_reaches_buff30'>

### Create Shrub and Forest Layers
 * Buffer study area by 60 meters to ensure no cells are missed in extraction
  * Extract NLCD raster by Study Watershed polygon mask and convert to Polygon

In [37]:
arcpy.env.workspace = outgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")
arcpy.env.extent = flowdirpath
arcpy.env.snapRaster = flowdirpath

studybuffname = region + "_buff60"
study_buff = arcpy.Buffer_analysis(studycopy,studybuffname,60, 'FULL', 'ROUND', 'ALL')

print ("Watershed Buffer complete", study_buff)
print ("")

from arcpy.sa import *
os.chdir(extractloc)
arcpy.env.workspace = extractloc
raster_list = arcpy.ListRasters()
start = datetime.datetime.now()
for raster in raster_list:
    if "ak_nlcd_2011" in raster:
        
        print (raster)
        print ("")
        
        desc = arcpy.Describe(raster)
        name = desc.baseName
        outname = temp_dir + "\\" + name + "_extract.tif"
        
        print ("Extracting raster ", name)
        print ("")
        
        extract = ExtractByMask(raster, study_buff)
        extract.save(outname)
        
        print ("Extraction Complete")
        print (extract)
        print ("")
                
stop = datetime.datetime.now()
elapsed = stop - start

print ("Process complete ", elapsed)
print ("")

arcpy.env.workspace = outgdb
nlcdpolyname = region + "_nlcd_poly"
nlcd_poly = arcpy.RasterToPolygon_conversion(extract,nlcdpolyname, 'NO_SIMPLIFY', 'Land_Cover')

print (nlcd_poly)

Watershed Buffer complete C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes\Kenai_Attributes_07022020\Kenai.gdb\Kenai_buff60

ak_nlcd_2011_landcover_1_15_15.img

Extracting raster  ak_nlcd_2011_landcover_1_15_15

Extraction Complete
C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes\Kenai_Attributes_07022020\ak_nlcd_2011_landcover_1_15_15_extract.tif

Process complete  0:00:01.761297

C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes\Kenai_Attributes_07022020\Kenai.gdb\Kenai_nlcd_poly


### Add and calc area in sq meters as check

In [38]:
arcpy.AddField_management(nlcd_poly,"Area_Sqm", 'DOUBLE')
arcpy.CalculateField_management(nlcd_poly, 'Area_Sqm','!shape.area@squaremeters!')
arcpy.AddField_management(nlcd_poly,"Land_Cover_Reclass", 'TEXT')


<Result 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\Watershed_Attributes\\Kenai\\Kenai_attributes\\Kenai_Attributes_07022020\\Kenai.gdb\\Kenai_nlcd_poly'>

### Add field and reclass cover types using following dictionary
 * Use update cursor and dictionary to reclassify

In [39]:
start = datetime.datetime.now()

d = {'Barren Land':'Barren',
'Cultivated Crops':'Crops',
'Deciduous Forest':'Forest',
'Mixed Forest':'Forest',
'Evergreen Forest':'Forest',
'Grassland/Herbaceous':'Herbaceous',
'Sedge/Herbaceous':'Herbaceous',
'Developed, High Intensity':'High intensity',
'Developed, Low Intensity':'Low intensity',
'Developed, Medium Intensity':'Medium intensity',
'Developed, Open Space':'Open space',
'Pasture/Hay':'Pasture',
'Perennial Ice/Snow':'Perennial Ice',
'Shrub/Scrub':'Shrub',
'Dwarf Shrub':'Shrub',
'Open Water':'Water',
'Emergent Herbaceous Wetlands':'Wetlands',
'Woody Wetlands':'Wetlands'}

fields = ['Land_Cover','Land_Cover_Reclass']
with arcpy.da.UpdateCursor(nlcd_poly, (fields)) as rows:
    for row in rows:
        if row[0] not in d.keys():
            print ("{} not in list".format(row[0]))
        else:
            row[1] = d[row[0]]
            rows.updateRow(row)
    del row 
del rows

stop = datetime.datetime.now()
elapsed = stop - start

print ("Process complete ", elapsed)


Process complete  0:00:14.306883


### Create Glacier layer and calculate percent cover by RCA
 * Use <a href = "http://glims.colorado.edu/glacierdata/">GLIMS</a> for glacier data - may require manual download
  * Large dataset that needs to be converted to feature class, projected, and clipped to study area
    * Returned void poly error when initially clipped so these steps may resolve the issue.  Try repair geometry if it does not.
    * Create a raster layer with 1 for glacier and 0 for no-glacier. Use the flow accumulation tool to get additive value for upstream contributing area for each RCA and then divide by flow accumulation value for each RCA.

In [40]:
start = datetime.datetime.now()
for dirpath, dirnames, filenames in arcpy.da.Walk(extractloc, datatype = "FEATURECLASS"):  
    for filename in filenames:  
        if "glims_polygons" in filename:
            glacierdata = os.path.join(dirpath,filename)
            print (glacierdata)
            
glims_copy = arcpy.FeatureClassToFeatureClass_conversion(glacierdata,outgdb,"glims_data") #make a copy to output gdb so that projections match and hopefully avoid void poly error in clip

end = datetime.datetime.now()
elapsed = end-start
print ("Time to complete",elapsed)

print (glims_copy)

C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes\Kenai_Attributes_07022020\extracts\glims_download_56134\glims_polygons.shp
Time to complete 0:00:19.990724
C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes\Kenai_Attributes_07022020\Kenai.gdb\glims_data


### Need to repair geometry or the next process will fail due to incomplete void poly error
* Must repair geometry on entire dataset prior to clip or clip will fail


In [41]:
arcpy.env.workspace = outgdb
start= datetime.datetime.now()

print ("Begin process", start)

glarepair = arcpy.RepairGeometry_management(glacierdata,'DELETE_NULL') #repair void poly errors in original dataset 

end = datetime.datetime.now()
elapsed = end-start
print ("Time to complete",elapsed)

Begin process 2020-02-07 16:07:13.123022
Time to complete 0:00:48.462915


### Clip glacier to buffered study area
 * buffered by 60 meters to ensure no data are missed

In [42]:
arcpy.env.workspace = outgdb
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

start= datetime.datetime.now()

print ("Begin process", start)
print("")

glaclip = arcpy.Clip_analysis(glarepair,study_buff,"glims_clip") #Clip glacier data to buffered study area

print (glaclip)
print("")

end = datetime.datetime.now()
elapsed = end-start
print ("Time to complete",elapsed)

Begin process 2020-02-07 16:08:01.603889

C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes\Kenai_Attributes_07022020\Kenai.gdb\glims_clip

Time to complete 0:00:21.547570


### Convert Glacier Polygon to raster

In [43]:
arcpy.env.workspace = temp_dir
arcpy.env.snapRaster = flowdirpath 
arcpy.env.extent = flowdirpath
arcpy.env.cellSize = flowdirpath

start = datetime.datetime.now()

print ("Begin process", start)
print("")

glimsrast = arcpy.PolygonToRaster_conversion(glaclip, 'glac_stat',"glims_poly2rast.tif","","",5) # convert polygon to raster and match cell size
arcpy.CalculateStatistics_management(glimsrast)

print(glimsrast)
print("")

stop = datetime.datetime.now()
elapsed = stop - start

print ("Process complete ", elapsed)

Begin process 2020-02-07 16:08:23.168416

C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes\Kenai_Attributes_07022020\glims_poly2rast.tif

Process complete  0:00:08.303975


### Convert Null values in glacier raster to 0

In [44]:
from arcpy.sa import *
arcpy.env.workspace = temp_dir
arcpy.env.snapRaster = flowdirpath
arcpy.env.extent = flowdirpath

start = datetime.datetime.now()

print ("Begin process", start)
print("")

glimsname = temp_dir + "\\" + region + "_glims_raster.tif"
glimscon = Con(IsNull(glimsrast),0,glimsrast) # Convert all null values to 0
glimscon.save(glimsname)

print(glimscon)
print ("")

stop = datetime.datetime.now()
elapsed = stop - start

print ("Process complete ", elapsed)

Begin process 2020-02-07 16:08:31.488348

C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes\Kenai_Attributes_07022020\Kenai_glims_raster.tif

Process complete  0:00:14.585024


### Create glacially weighted flow acc raster


In [45]:
arcpy.env.workspace = temp_dir
arcpy.env.snapRaster = flowdirpath
arcpy.env.extent = flowdirpath
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

start = datetime.datetime.now()

print ("Begin process", start)
print("")

glimsw8name = temp_dir + "\\" + region + "_glacial_weighted_flowacc.tif"
glimsw8flow = FlowAccumulation(flowdirpath, glimscon, 'FLOAT', 'D8') # create glacially weighted flow accumulation
glimsw8flow.save(glimsw8name)

print(glimsw8flow)
print ("")

stop = datetime.datetime.now()
elapsed = stop - start

print ("Process complete ", elapsed)
print ("")
print ("Calculate Percent Glacier along stream network")
print ("")

start = datetime.datetime.now()

glimspername = temp_dir + "\\" + region + "stream_per_glacier.tif"                
glimsper = SetNull(streamflow,((glimsw8flow / flowaccpath)*100 ),"Value = 0") #Limit output to stream network
glimsper.save(glimspername)

ExtractMultiValuesToPoints(outcopy,[[glimsper, "glacier_per"]])

stop = datetime.datetime.now()
elapsed = stop - start

print ("Process complete ", elapsed)
print(glimsper)

arcpy.JoinField_management(finalrca,"rca_id",outcopy, "rca_id","glacier_per" )

Begin process 2020-02-07 16:08:46.097309

C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes\Kenai_Attributes_07022020\Kenai_glacial_weighted_flowacc.tif

Process complete  0:01:12.177864

Calculate Percent Glacier along stream network

Process complete  0:01:12.544677
C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes\Kenai_Attributes_07022020\Kenaistream_per_glacier.tif


<Result 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\Watershed_Attributes\\Kenai\\Kenai_attributes\\Kenai_Attributes_07022020\\Kenai.gdb\\kenai_rcas_attributed_07022020'>

### Tabulate intersection between 30m buffered river segments (defined by nhdplusid or reachid) and nlcd polygon created above
 
 * Table output of land cover percent and total area for each land cover type within the buffered river segment 

In [46]:
for field in arcpy.ListFields(finalrca):
    print (field.name)

OBJECTID
Shape
rca_id
Shape_Length
Shape_Area
Join_Count
TARGET_FID
pointid
grid_code
rca_id_1
HUC12
Name
K_r
K_s
K_p
CH_r
CH_s
CH_p
CO_r
CO_s
CO_p
P_r
P_s
P_p
S_r
S_s
S_p
MIN
MAX
MEAN
Flow_Accumulation
Mean_Elev_Contributing_Area
glacier_per


In [47]:
start = datetime.datetime.now()
arcpy.env.workspace = outgdb
tab2name = "Reclassified_" + tabtablename
tab_int2 = arcpy.TabulateIntersection_analysis(riv_buff30, identifierfield, nlcd_poly,tab2name, ['Land_Cover_Reclass'], 'Area_Sqm')

print (tab_int2)
print ("")

stop = datetime.datetime.now()
elapsed = stop - start

print ("Process complete ", elapsed)

C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes\Kenai_Attributes_07022020\Kenai.gdb\Reclassified_kenai_rca_reaches_NLCD_Tab_Int

Process complete  0:01:54.098093


In [48]:

# start = datetime.datetime.now()
# arcpy.env.workspace = outgdb
# tab_int = arcpy.TabulateIntersection_analysis(riv_buff30, identifierfield, nlcd_poly,tabtablename, ['Land_Cover','Land_Cover_Reclass'], 'Area_Sqm')

# print (tab_int)
# print ("")

# stop = datetime.datetime.now()
# elapsed = stop - start

# print ("Process complete ", elapsed)

### Create Identity and dissolve to check against tabulate intersection



In [49]:
# start = datetime.datetime.now()
# arcpy.env.workspace = outgdb
# riv_nlcd_id = arcpy.Identity_analysis(riv_buff30, nlcd_poly,identname, 'ALL')
# end = datetime.datetime.now()
# elapsed = end-start

# print ("Time to complete",elapsed)
# print (riv_nlcd_id)
# print ("")

# idcount = arcpy.GetCount_management(riv_nlcd_id)

# print (idcount)

### Dissolve Identity
 * Keep ID, landcover, reclass, and buffered area
 * Add/calculate area in sqm and percent cover

In [50]:
# start = datetime.datetime.now()
# arcpy.env.workspace = outgdb
# disname = identname + "_All_Land_Dissolve"

# print("Begin Dissolve", start)
# print("")

# riv_nlcd_i0.d_dis = arcpy.Dissolve_management(riv_nlcd_id,disname, [identifierfield,'Land_Cover','Land_Cover_Reclass','Buff_Area_Sqm'], '', 'MULTI_PART', 'DISSOLVE_LINES')
# arcpy.AddField_management(riv_nlcd_id_dis,"Percent_Cover", 'DOUBLE')
# arcpy.AddField_management(riv_nlcd_id_dis,'Shape_Area_Sqm', 'DOUBLE')
# arcpy.CalculateField_management(riv_nlcd_id_dis, 'Shape_Area_Sqm','!shape.area@squaremeters!')
# percent_calc = "!Shape_Area_Sqm!/!Buff_Area_Sqm!*100"
# arcpy.CalculateField_management(riv_nlcd_id_dis, 'Percent_Cover',percent_calc)
# end = datetime.datetime.now()
# elapsed = end-start

# print ("Time to complete",elapsed)
# print (riv_nlcd_id_dis)
# print ("")

# disscount = arcpy.GetCount_management(riv_nlcd_id_dis)

# print (disscount)


In [51]:
# start = datetime.datetime.now()
# arcpy.env.workspace = outgdb
# disname2 = identname + "_Land_Reclass_Dissolve"

# print("Begin Dissolve", start)
# print("")

# riv_nlcd_id_dis2 = arcpy.Dissolve_management(riv_nlcd_id,disname2, [identifierfield, 'Land_Cover_Reclass','Buff_Area_Sqm'], '', 'MULTI_PART', 'DISSOLVE_LINES')
# arcpy.AddField_management(riv_nlcd_id_dis2,"Percent_Cover", 'DOUBLE')
# arcpy.AddField_management(riv_nlcd_id_dis2,'Shape_Area_Sqm', 'DOUBLE')
# arcpy.CalculateField_management(riv_nlcd_id_dis2, 'Shape_Area_Sqm','!shape.area@squaremeters!')
# percent_calc = "!Shape_Area_Sqm!/!Buff_Area_Sqm!*100"
# arcpy.CalculateField_management(riv_nlcd_id_dis2, 'Percent_Cover',percent_calc)
# end = datetime.datetime.now()
# elapsed = end-start

# print ("Time to complete",elapsed)
# print (riv_nlcd_id_dis2)
# print ("")

# disscount = arcpy.GetCount_management(riv_nlcd_id_dis2)

# print (disscount)

### Convert table to numpy array

In [52]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format # only display 2 decimal places
field_list = []
for field in arcpy.ListFields(tab_int2):
    field_list.append(field.name)
tabint_arr = arcpy.da.TableToNumPyArray(tab_int2,field_list)
df = pd.DataFrame(tabint_arr)
df

,OBJECTID,reachid,Land_Cover_Reclass,Area_Sqm,AREA,PERCENTAGE
0,1,1,Barren,11243.60,11243.60,74.53
1,2,1,Forest,3.97,3.97,0.03
2,3,1,Shrub,3258.09,3258.09,21.60
3,4,1,Water,580.87,580.87,3.85
4,5,2,Barren,11453.72,11453.72,58.41
...,...,...,...,...,...,...
8839,8840,3543,Shrub,63870.63,63870.63,21.20
8840,8841,3543,Water,76409.43,76409.43,25.36
8841,8842,3544,Forest,24346.13,24346.13,12.11
8842,8843,3544,Shrub,172529.44,172529.44,85.80


### List of unique land cover classes from tabulate intersection
 * All land cover types that are present in the study area

In [53]:
lancov = df['Land_Cover_Reclass'].unique()
print (sorted(lancov))

['Barren', 'Crops', 'Forest', 'Herbaceous', 'High intensity', 'Low intensity', 'Medium intensity', 'Open space', 'Pasture', 'Perennial Ice', 'Shrub', 'Water', 'Wetlands']


### Reshape dataframe and set index
 * Easier to read

In [54]:
df2 = df.drop(["OBJECTID","AREA"],axis=1)
df2 = df2.set_index([identifierfield,'Land_Cover_Reclass'])
df2


Area_Sqm  PERCENTAGE
reachid Land_Cover_Reclass                      
1       Barren              11243.60       74.53
        Forest                  3.97        0.03
        Shrub                3258.09       21.60
        Water                 580.87        3.85
2       Barren              11453.72       58.41
...                              ...         ...
3543    Shrub               63870.63       21.20
        Water               76409.43       25.36
3544    Forest              24346.13       12.11
        Shrub              172529.44       85.80
        Wetlands             4200.84        2.09

[8844 rows x 2 columns]

### Compare Dissolve to Table

In [55]:
# import pandas as pd
# pd.set_option('display.max_columns', None)  
# from arcgis.features import GeoAccessor, GeoSeriesAccessor
# from arcgis import GIS
# gis = GIS()
# sdf = pd.DataFrame.spatial.from_featureclass(riv_nlcd_id_dis2)
# sdf[[identifierfield,'Land_Cover_Reclass','Buff_Area_Sqm','Shape_Area_Sqm','Percent_Cover']].sort_values(by=[identifierfield])


### Pivot Tab Intersection table and join to rca reaches
 * Turn land cover classes stored in rows to fields

In [56]:
for field in arcpy.ListFields(reachcopy):
    print (field.name)
    

OBJECTID_1
Shape
OBJECTID
reachid
Shape_Length
Z_Min
Z_Max
Z_Mean
SLength
reach_length
reach_slope


In [57]:
arcpy.env.workspace = outgdb
pivname = region + "Landcover_Pivot_Table"
tabpivot = arcpy.PivotTable_management(tab_int2,"reachid","Land_Cover_Reclass","PERCENTAGE",pivname)
arcpy.JoinField_management(reachcopy,"reachid",tabpivot,"reachid",["Forest","Shrub","Wetlands"])
arcpy.AddField_management(reachcopy,"Riparian", 'TEXT',"","",250,"Percent riparian (forest + shrub) cover in 30 meter buffer surrounding reach")

fieldList = ["Forest","Shrub","Wetlands"]
with arcpy.da.UpdateCursor(reachcopy, fieldList) as cursor:
        for row in cursor:
            Urow = row
            for i in range (len(fieldList)):
                if Urow[i] == None:
                    Urow[i] = 0
                
            cursor.updateRow(Urow)

print ("Processing complete")
print("")

ripcalc = "!Forest!+!Shrub!"
arcpy.CalculateField_management(reachcopy,"Riparian", ripcalc)

for field in arcpy.ListFields(reachcopy):
    print (field.name)

Processing complete

OBJECTID_1
Shape
OBJECTID
reachid
Shape_Length
Z_Min
Z_Max
Z_Mean
SLength
reach_length
reach_slope
Forest
Shrub
Wetlands
Riparian



### Rename Fields and delete unecessary joins
 * create dictionary to store oldname/new name alias combos

In [61]:
rcaDict = {'rca_id': ('rca_id', 'Unique ID associated with each reach contributing area (rca)'),
'HUC12': ('HUC12', '12th digit hydrologic unit code associated with rca'),
'Name': ('HUC_name', 'name of 12th digit hydrologic unit code associated with rca'),
'K_r': ('K_r', 'Presence/absence of Chinook salmon rearing habitat in rca'),
'K_s': ('K_s', 'Presence/absence of Chinook salmon spawning habitat in rca'),
'K_p': ('K_p', 'Presence/absence of Chinook salmon in rca'),
'CH_r': ('CH_r', 'Presence/absence of chum salmon rearing habitat in rca'),
'CH_s': ('CH_s', 'Presence/absence of chum salmon spawning habitat in rca'),
'CH_p': ('CH_p', 'Presence/absence of chum salmon in rca'),
'CO_r': ('CO_r', 'Presence/absence of coho salmon rearing habitat in rca'),
'CO_s': ('CO_s', 'Presence/absence of coho salmon spawning habitat in rca'),
'CO_p': ('CO_p', 'Presence/absence of coho salmon in rca'),
'P_r': ('P_r', 'Presence/absence of pink salmon rearing habitat in rca'),
'P_s': ('P_s', 'Presence/absence of pink salmon spawning habitat in rca'),
'P_p': ('P_p', 'Presence/absence of pink salmon in rca'),
'S_r': ('S_r', 'Presence/absence of sockeye salmon rearing habitat in rca'),
'S_s': ('S_s', 'Presence/absence of sockeye salmon spawning habitat in rca'),
'S_p': ('S_p', 'Presence/absence of sockeye salmon in rca'),
'MIN': ('rca_elev_min', 'Minimum rca elevation '),
'MAX': ('rca_elev_max', 'Maximum rca elevation '),
'MEAN': ('rca_elev_mn', 'Mean rca elevation '),
'Flow_Accumulation': ('flowacc', 'Number of upstream cells draining to rca pour point'),
'Mean_Elev_Contributing_Area': ('ca_elev_mn', 'Mean elevation of upstream contributing area draining to rca pour point')}

reachDict = {'reachid': ('reach_id', 'Unique ID associated with each confluence to confluence reach '), 'Forest': ('Forest', 'Percent forest cover in 30 meter buffer surrounding reach'),
                         'Shrub': ('Shrub', 'Percent shrub cover in 30 meter buffer surrounding reach'),
                         'Wetlands': ('Wetland', 'Percent wetland cover in 30 meter buffer surrounding reach')}
tempDict = {}

outletDict = {'rca_id': ('rca_id', 'Unique ID associated with each reach contributing area (rca)'),
'HUC12': ('HUC12', '12th digit hydrologic unit code associated with rca'),
'Name': ('HUC_name', 'name of 12th digit hydrologic unit code associated with rca'),
'K_r': ('K_r', 'Presence/absence of Chinook salmon rearing habitat in rca'),
'K_s': ('K_s', 'Presence/absence of Chinook salmon spawning habitat in rca'),
'K_p': ('K_p', 'Presence/absence of Chinook salmon in rca'),
'CH_r': ('CH_r', 'Presence/absence of chum salmon rearing habitat in rca'),
'CH_s': ('CH_s', 'Presence/absence of chum salmon spawning habitat in rca'),
'CH_p': ('CH_p', 'Presence/absence of chum salmon in rca'),
'CO_r': ('CO_r', 'Presence/absence of coho salmon rearing habitat in rca'),
'CO_s': ('CO_s', 'Presence/absence of coho salmon spawning habitat in rca'),
'CO_p': ('CO_p', 'Presence/absence of coho salmon in rca'),
'P_r': ('P_r', 'Presence/absence of pink salmon rearing habitat in rca'),
'P_s': ('P_s', 'Presence/absence of pink salmon spawning habitat in rca'),
'P_p': ('P_p', 'Presence/absence of pink salmon in rca'),
'S_r': ('S_r', 'Presence/absence of sockeye salmon rearing habitat in rca'),
'S_s': ('S_s', 'Presence/absence of sockeye salmon spawning habitat in rca'),
'S_p': ('S_p', 'Presence/absence of sockeye salmon in rca'),
'MIN': ('rca_elev_min', 'Minimum rca elevation '),
'MAX': ('rca_elev_max', 'Maximum rca elevation '),
'MEAN': ('rca_elev_mn', 'Mean rca elevation '),
'Flow_Accumulation': ('flowacc', 'Number of upstream cells draining to rca pour point'),
'Mean_Elev_Contributing_Area': ('ca_elev_mn', 'Mean elevation of upstream contributing area draining to rca pour point')}


In [62]:
for field in arcpy.ListFields(reachcopy):
    keyval = field.name
    if keyval in reachDict:
        newname = reachDict[keyval][0]
        newalias = reachDict[keyval][1]
               
        arcpy.AlterField_management(reachcopy, keyval, newname, newalias)

for field in arcpy.ListFields(finalrca):       
    keyval = field.name
    if keyval in rcaDict:
        newname = rcaDict[keyval][0]
        newalias = rcaDict[keyval][1]

        arcpy.AlterField_management(finalrca, keyval, newname, newalias)
        
for field in arcpy.ListFields(outcopy):
    keyval = field.name
    if keyval in outletDict:
        newname = outletDict[keyval][0]
        newalias = outletDict[keyval][1]
           
        arcpy.AlterField_management(outcopy, keyval, newname, newalias)
        
print ("Field renaming complete")


Field renaming complete


### Delete unecessary fields



In [64]:

rcadrops = ["Join_Count","TARGET_FID","pointid","grid_code","rca_id_1"]
reachdrops =["Z_Min","Z_Max","Z_Mean","SLength"] 

for field in rcadrops:
    arcpy.DeleteField_management(finalrca,field)

for field in reachdrops:
    arcpy.DeleteField_management(reachcopy,field)

for field in rcadrops:
    arcpy.DeleteField_management(outcopy,field)
    
print ("Fields cleaned")

Fields cleaned


### Check extent of Rasters against input flow direction

In [67]:
rasterlist = [streamrast, dem_copy, glimsw8flow, elevw8flow, flowacc]

desc = arcpy.Describe(flowdirpath)
ext1 =desc.extent # get extent of flow dir
xm = ext1.XMin
ym = ext1.YMin
xM = ext1.XMax
yM = ext1.YMax
cS1 = desc.meanCellHeight
spr = desc.SpatialReference.name

print ("Flow Direction Raster Information")
print("")
print("Raster name:      %s" % desc.name)
print("Projection:      %s" % desc.SpatialReference.name)
print("Compression Type: %s" % desc.compressionType)
print("Raster Format:    %s" % desc.format)
print("Height: %d" % desc.height)
print("Width:  %d" % desc.width)
print("Cellsize:  %f" % desc.meanCellHeight)
print("Integer Raster: %s" % desc.isInteger)
print("Raster stats: min = {:,.2f} max = {:,.2f} mean = {:,.2f}".format(raster_min,raster_max,raster_mean))
print (extent)
print ("")

import pandas as pd    
data = []
for r in rasterlist:
    rdesc = arcpy.Describe(r)
    ext = rdesc.extent
    cS = rdesc.meanCellHeight
    rn = rdesc.name
    data.append([rn, ext.XMin, ext.YMin, ext.XMax, ext.YMax,cS,spr])



df = pd.DataFrame(data=data, columns=["raster",'XMin','YMin','XMax','YMax', 'Cell Size','Spatial Reference'])
df2 = df.set_index(['raster'])
df2




Flow Direction Raster Information

Raster name:      Kenai_StrBrn_d8flowdir.tif
Projection:      NAD_1983_Alaska_Albers
Compression Type: LZW
Raster Format:    TIFF
Height: 18755
Width:  27650
Cellsize:  5.000000
Integer Raster: True
Raster stats: min = 0.00 max = 1,974.00 mean = 521.31
147477.63147271 1116160.30748062 285727.63147271 1209935.30748062 NaN NaN NaN NaN



,XMin,YMin,XMax,YMax,Cell Size,Spatial Reference
raster,,,,,,
Kenai_StrBrn_src.tif,147477.63,1116160.31,285727.63,1209935.31,5.00,NAD_1983_Alaska_Albers
Kenai_StrBrn_2Int.tif,147477.63,1116160.31,285727.63,1209935.31,5.00,NAD_1983_Alaska_Albers
Kenai_glacial_weighted_flowacc.tif,147477.63,1116160.31,285727.63,1209935.31,5.00,NAD_1983_Alaska_Albers
Kenai_elevation_weighted_flowacc.tif,147477.63,1116160.31,285727.63,1209935.31,5.00,NAD_1983_Alaska_Albers
Kenai_flow_accumulation_07022020.tif,147477.63,1116160.31,285727.63,1209935.31,5.00,NAD_1983_Alaska_Albers



### Check that all extents, cellsizes, and projections match
 * Values should be true (match flowdir raster)

In [76]:
rasterlist = [streamrast, dem_copy, glimsw8flow, elevw8flow, flowacc]

desc = arcpy.Describe(flowdirpath)
ext1 =desc.extent # get extent of flow dir
xm = ext1.XMin
ym = ext1.YMin
xM = ext1.XMax
yM = ext1.YMax
cS1 = desc.meanCellHeight
spr = desc.SpatialReference.name

print ("Flow Direction Raster Information")
print("")
print("Raster name:      %s" % desc.name)
print("Projection:      %s" % desc.SpatialReference.name)
print("Compression Type: %s" % desc.compressionType)
print("Raster Format:    %s" % desc.format)
print("Height: %d" % desc.height)
print("Width:  %d" % desc.width)
print("Cellsize:  %f" % desc.meanCellHeight)
print("Integer Raster: %s" % desc.isInteger)
print("Raster stats: min = {:,.2f} max = {:,.2f} mean = {:,.2f}".format(raster_min,raster_max,raster_mean))
print (extent)
print ("")

import pandas as pd    
data = []
for r in rasterlist:
    rdesc = arcpy.Describe(r)
    ext = rdesc.extent
    cS = rdesc.meanCellHeight
    rn = rdesc.name
    data.append([rn, ext.XMin, ext.YMin, ext.XMax, ext.YMax,cS,spr])



df = pd.DataFrame(data=data, columns=["raster",'XMin','YMin','XMax','YMax', 'Cell Size','Spatial Reference'])
df2 = df.set_index(['raster'])
df2.isin([xm,ym,xM,yM,cS1,spr])

Flow Direction Raster Information

Raster name:      Kenai_StrBrn_d8flowdir.tif
Projection:      NAD_1983_Alaska_Albers
Compression Type: LZW
Raster Format:    TIFF
Height: 18755
Width:  27650
Cellsize:  5.000000
Integer Raster: True
Raster stats: min = 0.00 max = 1,974.00 mean = 521.31
147477.63147271 1116160.30748062 285727.63147271 1209935.30748062 NaN NaN NaN NaN



,XMin,YMin,XMax,YMax,Cell Size,Spatial Reference
raster,,,,,,
Kenai_StrBrn_src.tif,True,True,True,True,True,True
Kenai_StrBrn_2Int.tif,True,True,True,True,True,True
Kenai_glacial_weighted_flowacc.tif,True,True,True,True,True,True
Kenai_elevation_weighted_flowacc.tif,True,True,True,True,True,True
Kenai_flow_accumulation_07022020.tif,True,True,True,True,True,True


In [77]:
rastlist = [streamrast, flowdirpath, dem_copy, glimsw8flow, elevw8flow, flowacc]
extents = []

for raster in rastlist:
    desc = arcpy.Describe(raster)
        
    raster_min = arcpy.Raster(raster).minimum
    raster_max = arcpy.Raster(raster).maximum
    raster_mean = arcpy.Raster(raster).mean
    extent = arcpy.Raster(raster).extent
    extents.append(extent)
    
    print("Raster name:      %s" % desc.name)
    print("Projection:      %s" % desc.SpatialReference.name)
    print("Compression Type: %s" % desc.compressionType)
    print("Raster Format:    %s" % desc.format)
    print("Height: %d" % desc.height)
    print("Width:  %d" % desc.width)
    print("Cellsize:  %f" % desc.meanCellHeight)
    print("Integer Raster: %s" % desc.isInteger)
    print("Raster stats: min = {:,.2f} max = {:,.2f} mean = {:,.2f}".format(raster_min,raster_max,raster_mean))
    print (extent)
    print ("")
    

Raster name:      Kenai_StrBrn_src.tif
Projection:      NAD_1983_Alaska_Albers
Compression Type: LZW
Raster Format:    TIFF
Height: 18755
Width:  27650
Cellsize:  5.000000
Integer Raster: True
Raster stats: min = 0.00 max = 1.00 mean = 0.01
147477.63147271 1116160.30748062 285727.63147271 1209935.30748062 NaN NaN NaN NaN

Raster name:      Kenai_StrBrn_d8flowdir.tif
Projection:      NAD_1983_Alaska_Albers
Compression Type: LZW
Raster Format:    TIFF
Height: 18755
Width:  27650
Cellsize:  5.000000
Integer Raster: True
Raster stats: min = 1.00 max = 128.00 mean = 33.07
147477.63147271 1116160.30748062 285727.63147271 1209935.30748062 NaN NaN NaN NaN

Raster name:      Kenai_StrBrn_2Int.tif
Projection:      NAD_1983_Alaska_Albers
Compression Type: LZW
Raster Format:    TIFF
Height: 18755
Width:  27650
Cellsize:  5.000000
Integer Raster: True
Raster stats: min = 0.00 max = 1,974.00 mean = 521.31
147477.63147271 1116160.30748062 285727.63147271 1209935.30748062 NaN NaN NaN NaN

Raster name:

### Copy to Network GDB

In [78]:
arcpy.env.workspace = networkgdbfd

arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("Alaska Albers Equal Area Conic")
spref = arcpy.SpatialReference("Alaska Albers Equal Area Conic")

exportfcs= []

exportfcs = [tempfinal,finalrca,outcopy]

for fc in exportfcs:
    desc = arcpy.Describe(fc)
    print("Copying...", desc.name)
    print("")
    
    arcpy.CopyFeatures_management(fc,desc.name)

reachdesc = arcpy.Describe(reachcopy)

print ("Copying", reachdesc.name)
print ("")

reachcopyname = reachdesc.name + "_attributed_" + str(time_stamp)
arcpy.CopyFeatures_management(reachcopy,reachcopyname)

print ("All features copied")

Copying... kenai_rcas_temp_sites_attributed_07022020

Copying... kenai_rcas_attributed_07022020

Copying... Kenai_rca_outlets_attributed_07022020

Copying kenai_rca_reaches

All features copied


### Copy Rasters
 * Tool will throw validation error but process completes


In [83]:
exportrasters = [flowdirpath, glimscon, flowacc, dem_copy]

for r in exportrasters:
       
    print("Copying ...", r)
    print("")
    
    arcpy.RasterToGeodatabase_conversion (r,networkgdb)

print ("Raster copy complete")    

Copying ... C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes\Kenai_Attributes_07022020\Kenai_StrBrn_d8flowdir.tif



AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

Copying ... C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes\Kenai_Attributes_07022020\Kenai_glims_raster.tif



AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

...

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

Copying ... C:\Users\dwmerrigan\Documents\GitHub\Watershed_Attributes\Kenai\Kenai_attributes\Kenai_Attributes_07022020\Kenai_StrBrn_2Int.tif



AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

Raster copy complete


### Check that exported rasters match original flow dir

In [84]:
arcpy.env.workspace = networkgdb
rasterlist = []
for raster in arcpy.ListRasters():
    rasterlist.append(raster)

print (rasterlist)
print("")
    
desc = arcpy.Describe(flowdirpath)
ext1 =desc.extent # get extent of flow dir
xm = ext1.XMin
ym = ext1.YMin
xM = ext1.XMax
yM = ext1.YMax
cS1 = desc.meanCellHeight
spr = desc.SpatialReference.name

print ("Flow Direction Raster Information")
print("")
print("Raster name:      %s" % desc.name)
print("Projection:      %s" % desc.SpatialReference.name)
print("Compression Type: %s" % desc.compressionType)
print("Raster Format:    %s" % desc.format)
print("Height: %d" % desc.height)
print("Width:  %d" % desc.width)
print("Cellsize:  %f" % desc.meanCellHeight)
print("Integer Raster: %s" % desc.isInteger)
print("Raster stats: min = {:,.2f} max = {:,.2f} mean = {:,.2f}".format(raster_min,raster_max,raster_mean))
print (extent)
print ("")

import pandas as pd    
data = []
for r in rasterlist:
    rdesc = arcpy.Describe(r)
    ext = rdesc.extent
    cS = rdesc.meanCellHeight
    rn = rdesc.name
    data.append([rn, ext.XMin, ext.YMin, ext.XMax, ext.YMax,cS,spr])

df = pd.DataFrame(data=data, columns=["raster",'XMin','YMin','XMax','YMax', 'Cell Size','Spatial Reference'])
df2 = df.set_index(['raster'])
df2.isin([xm,ym,xM,yM,cS1,spr])

['Kenai_StrBrn_d8flowdir', 'Kenai_glims_raster', 'Kenai_flow_accumulation_07022020', 'Kenai_StrBrn_2Int', 'Kenai_StrBrn_d8flowdir_1', 'Kenai_glims_raster_1', 'Kenai_flow_accumulation_07022020_1', 'Kenai_StrBrn_2Int_1']

Flow Direction Raster Information

Raster name:      Kenai_StrBrn_d8flowdir.tif
Projection:      NAD_1983_Alaska_Albers
Compression Type: LZW
Raster Format:    TIFF
Height: 18755
Width:  27650
Cellsize:  5.000000
Integer Raster: True
Raster stats: min = 0.00 max = 216,265,392.00 mean = 21,693.79
147477.63147271 1116160.30748062 285727.63147271 1209935.30748062 NaN NaN NaN NaN



,XMin,YMin,XMax,YMax,Cell Size,Spatial Reference
raster,,,,,,
Kenai_StrBrn_d8flowdir,True,True,True,True,True,True
Kenai_glims_raster,True,True,True,True,True,True
Kenai_flow_accumulation_07022020,True,True,True,True,True,True
Kenai_StrBrn_2Int,True,True,True,True,True,True
Kenai_StrBrn_d8flowdir_1,True,True,True,True,True,True
Kenai_glims_raster_1,True,True,True,True,True,True
Kenai_flow_accumulation_07022020_1,True,True,True,True,True,True
Kenai_StrBrn_2Int_1,True,True,True,True,True,True


### Compare output feature classes to inputs

In [85]:
arcpy.env.workspace = networkgdbfd

ofclist = []
ofclist = [tempsites,inrca,instreams]


for fc in arcpy.ListFeatureClasses():
    fcd = arcpy.Describe(fc)
    count = arcpy.GetCount_management(fc)
    print (fcd.name, "has ", count, " features")
    
print("")

for ofc in ofclist:
    ofcd = arcpy.Describe(ofc)
    count = arcpy.GetCount_management(ofc)
    print (ofcd.name, "has ", count, " features")


kenai_rca_confluence has  1771  features
kenai_waterbody_NHD has  1229  features
kenai_streams_NHDP has  5738  features
kenai_rcas_temp_sites_attributed_07022020 has  28  features
kenai_rcas_attributed_07022020 has  3325  features
Kenai_rca_outlets_attributed_07022020 has  3326  features
kenai_rca_reaches_attributed_07022020 has  3544  features

tempsites_Kenai has  28  features
kenai_rcas has  3325  features
kenai_rca_reaches has  3544  features


### Delete Identical outlet in Kenai
 * Not sure why this did not work in code above, the count even shows it deleted in this [section of code](#deleteid)
 
 

In [86]:
arcpy.env.workspace = networkgdbfd

for fc in arcpy.ListFeatureClasses():
    fcd = arcpy.Describe(fc)
    if "outlets" in fcd.name:
        
        count = arcpy.GetCount_management(fc)
        print (fcd.name, "has ", count, " features")
        print ("")
        
        outletfinalpath = os.path.join(fcd.path,fcd.name)
        
        arcpy.DeleteIdentical_management(fc, 'rca_id')
        
        count2 = arcpy.GetCount_management(fc)
        
        print ("count after delete id", count2)
        print("")
        
        


Kenai_rca_outlets_attributed_07022020 has  3326  features

count after delete id 3325



<a id = 'bottom'></a>

Jump to [top](#top)